# Data Processing

In [82]:
import os
import pandas as pd

import importlib
import pickle

In [ ]:
import DataProcessor
importlib.reload(DataProcessor)

In [ ]:
data_processor = DataProcessor.DataProcessor()

In [ ]:
df = data_processor.get_euler_dataframe()
# save df
df.to_pickle("/data/euler_df.pkl")

In [ ]:
# load df from pickle
df = pd.read_pickle("./data/euler_df.pkl")

In [ ]:
df

# Test CodeT5

In [ ]:
from transformers import T5ForConditionalGeneration, RobertaTokenizer

In [ ]:
tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-small')
model = T5ForConditionalGeneration.from_pretrained('Salesforce/codet5-small')

text = "def greet(user): print(f'hello <extra_id_0>!')"
input_ids = tokenizer(text, return_tensors="pt").input_ids

# simply generate a single sequence
generated_ids = model.generate(input_ids, max_length=10)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
# this prints "user: {user.name}"

In [ ]:
tokenizer(text, return_tensors="pt")

In [ ]:
generated_ids

In [ ]:
tokenizer.decode(generated_ids[0][3])

In [ ]:
import ProblemSetReader
importlib.reload(ProblemSetReader)

In [ ]:
problem_set_reader = ProblemSetReader.ProblemSetReader()

In [ ]:
question, answer, code = problem_set_reader.get_problem_set(1)

In [ ]:
question

In [ ]:
input_ids = tokenizer(question, return_tensors="pt").input_ids
generated_ids = model.generate(input_ids, max_length=500)
print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))

# Test CodeGen

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM

In [69]:
size_str = "2B"
tokenizer = AutoTokenizer.from_pretrained(f"Salesforce/codegen-{size_str}-mono")
model = AutoModelForCausalLM.from_pretrained(f"Salesforce/codegen-{size_str}-mono",
                                             output_hidden_states=False)

In [76]:
text = problem_set_reader.get_question(50)+"\"\"\""
input_ids = tokenizer(text, return_tensors="pt").input_ids

completion = model.generate(**tokenizer(text, return_tensors="pt"), 
                            max_length=512)
print(tokenizer.decode(completion[0][:len(text)]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


The prime $41$, can be written as the sum of six consecutive primes:
$$41 = 2 + 3 + 5 + 7 + 11 + 13.$$
This is the longest sum of consecutive primes that adds to a prime below one-hundred.
The longest sum of consecutive primes below one-thousand that adds to a prime, contains $21$ terms, and is equal to $953$.
Which prime, below one-million, can be written as the sum of the most consecutive primes?
"""

from math import sqrt

def is_prime(n):
    if n == 1:
        return False
    for i in range(2, int(sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def solution(limit):
    primes = []
    for i in range(2, limit):
        if is_prime(i):
            primes.append(i)
    
    max_prime = 0
    max_prime_sum = 0
    for i in range(len(primes)):
        for j in range(i, len(primes)):
            sum = primes[i] + primes[j]
            if sum < limit:
                if is_prime(sum):
                    if sum > max_prime_sum:
                        max

In [79]:
len(input_ids)

1

In [81]:
print(tokenizer.decode(completion[0][len(input_ids[0]):][:-1]))



from math import sqrt

def is_prime(n):
    if n == 1:
        return False
    for i in range(2, int(sqrt(n)) + 1):
        if n % i == 0:
            return False
    return True

def solution(limit):
    primes = []
    for i in range(2, limit):
        if is_prime(i):
            primes.append(i)
    
    max_prime = 0
    max_prime_sum = 0
    for i in range(len(primes)):
        for j in range(i, len(primes)):
            sum = primes[i] + primes[j]
            if sum < limit:
                if is_prime(sum):
                    if sum > max_prime_sum:
                        max_prime_sum = sum
                        max_prime = primes[i] + primes[j]
    return max_prime

print(solution(1000000))



In [84]:
generated_path = os.path.join(os.getcwd(), "data", "generated")
if not os.path.isdir(generated_path):
    os.mkdir(generated_path)

In [85]:
idx = 50
generated_code_str = tokenizer.decode(completion[0][len(input_ids[0]):][:-1])

with open(os.path.join(generated_path, f"p{idx:03d}.py"), "w", encoding="utf-8") as f:
    f.write(generated_code_str)
    